<a href="https://colab.research.google.com/github/kavyajeetbora/modern_geospatial_stack/blob/master/use_cases/retail_store_analysis_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
!pip install -q streamlit
!npm install localtunnel
import urllib
!wget https://github.com/kavyajeetbora/modern_geospatial_stack/raw/master/use_cases/pop_rad_data.parquet -O pop_rad_data.parquet
!wget https://github.com/kavyajeetbora/modern_geospatial_stack/raw/master/use_cases/poi_accessibility.parquet -O poi_accessibility.parquet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 1.5.3 which is incompatible.
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.49s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm 

array([2.62570988e-01, 5.27274113e-01, 5.97956751e-01, 2.25667525e-01,
       4.34804734e-01, 3.87767868e-02, 2.52840757e-01, 4.39245265e-01,
       9.24207613e-03, 5.12447230e-01, 4.61433147e-02, 2.25864990e-01,
       8.33769551e-01, 7.51882843e-01, 7.49080414e-01, 6.83156089e-01,
       6.07692898e-01, 5.79163027e-01, 7.25065411e-01, 4.67725547e-01,
       3.20463697e-01, 2.91277149e-01, 2.61305203e-01, 6.14384952e-01,
       7.56628333e-01, 2.72074861e-01, 4.16663214e-01, 7.10628232e-01,
       2.29576078e-01, 4.44164831e-01, 2.62285439e-01, 6.49617609e-01,
       1.86894097e-01, 5.89768417e-01, 5.16377852e-01, 6.67664018e-01,
       5.20426820e-01, 5.77020403e-01, 6.12719808e-01, 8.33181518e-01,
       5.87572982e-01, 2.25588086e-01, 5.00964623e-02, 4.42721950e-02,
       5.60709497e-01, 8.71085434e-01, 1.96329804e-01, 3.45400222e-01,
       3.86494771e-01, 6.62109265e-01, 3.11439617e-01, 5.71262976e-01,
       3.16926805e-01, 5.98053745e-01, 4.39470969e-01, 4.33947266e-01,
      

In [108]:
%%writefile app.py
import streamlit as st
import pydeck as pdk
import pandas as pd
from matplotlib import colormaps
from sklearn.preprocessing import MinMaxScaler


## STREAMLIT CONFIGURATION
## --------------------------------------------------------------------------------##
st.set_page_config(page_title='Retail Store Location Analysis', page_icon='', layout="wide")

pop_rad_data = pd.read_parquet('pop_rad_data.parquet').fillna(0)
pop_rad_data['radiance'] = pop_rad_data['radiance'].astype(int)
poi_accessibility = pd.read_parquet('poi_accessibility.parquet').fillna(0)

## Create the overall suitability score

scaler = MinMaxScaler()

df = pop_rad_data[['hex8','radiance', 'population']].set_index('hex8').join(poi_accessibility[['hex8','poi_count', 'time_min']].set_index('hex8'))
df[['radiance', 'population','poi_count', 'time_min']] = scaler.fit_transform(df[['radiance', 'population','poi_count', 'time_min']])
df['suitablity_score'] = df.sum(axis=1)
df = df[['suitablity_score']].reset_index()
scaler = MinMaxScaler()
df['suitablity_score'] = (scaler.fit_transform(df['suitablity_score'].values.reshape(-1,1)).ravel()*100).astype(int)

def color_code_value(value, cmap):
    color = cmap(value)
    scaled_colors = list(map(lambda x: int(x * 255), color[:3]))
    scaled_alpha = int(color[3] * 100)

    scaled_colors += [scaled_alpha]
    return scaled_colors


def colormap_dataframe(df, value_col, cmap):

    xdf = df.copy()

    ## Scale the temperature values between 0-1; using MinMax Scaler
    xdf[f"norm_{value_col}"] = (xdf[value_col] - xdf[value_col].min()) / (
        xdf[value_col].max() - xdf[value_col].min()
    )
    xdf["color"] = xdf[f"norm_{value_col}"].apply(lambda x: color_code_value(x, cmap))
    xdf[["R", "G", "B", "A"]] = pd.DataFrame(xdf["color"].to_list())
    xdf = xdf.drop(["color", f"norm_{value_col}"], axis=1)

    ## formatting the temperature value upto 1 decimal place
    xdf[value_col] = xdf[value_col].round(1)

    return xdf


def create_h3_hex_layer(df, hex_col):

    # Define a layer to display on a map
    layer = pdk.Layer(
        "H3HexagonLayer",
        df,
        pickable=True,
        stroked=True,
        filled=True,
        extruded=False,
        get_hexagon=hex_col,
        get_fill_color="[R, G, B, A]",
    )

    return layer

hex_col = 'hex8'

## Radiance data
radiance = colormap_dataframe(pop_rad_data[['hex8', 'radiance']], value_col='radiance', cmap=colormaps['cividis'])
rad_layer = create_h3_hex_layer(radiance, hex_col)

# Population data
pop_age_sex = colormap_dataframe(pop_rad_data[['hex8', 'population', 'sum_male', 'sum_female']], value_col='population', cmap=colormaps['seismic'])
pop_age_sex_layer = create_h3_hex_layer(pop_age_sex, hex_col)

# Density of Amenity
poi_density = colormap_dataframe(poi_accessibility[['hex8', 'poi_count']], value_col='poi_count', cmap=colormaps['viridis'])
poi_density_layer = create_h3_hex_layer(poi_density, hex_col)

# Accessibility
accessibility = colormap_dataframe(poi_accessibility[['hex8', 'time_min']], value_col='time_min', cmap=colormaps['YlOrRd'])
accessibility_layer = create_h3_hex_layer(accessibility, hex_col)

## suitability score
score = colormap_dataframe(df[['hex8', 'suitablity_score']], value_col='suitablity_score', cmap=colormaps['YlOrRd'])
score_layer = create_h3_hex_layer(score, hex_col)

def generate_tooltip(text):
    tooltip = {
            "html": text,
            "style": {
                "backgroundColor": "#4CAF50",   # Green shade for background
                "color": "#FFFFFF",             # White for text color
                "border": "2px solid #4CAF50",  # Matching border color
                "borderRadius": "5px",          # Rounded corners
                "boxShadow": "2px 2px 10px rgba(0, 0, 0, 0.2)"  # Soft shadow effect
            }
        }

    return tooltip

options = {
    "Density of Amenities": (poi_density_layer, generate_tooltip("Amenities Count: {poi_count}")),
    "Public Transport Accessibility": (accessibility_layer, generate_tooltip("Pedestrian Accessibility: {time_min} mins")),
    "Population Density": (pop_age_sex_layer, generate_tooltip('''Population: {population} <br>
        Male: {sum_male} % <br>
        Female: {sum_female} %
        ''')),
    "Nighttime Lights": (rad_layer, generate_tooltip("Radiance: {radiance}")),
    "Suitability Score": (score_layer, generate_tooltip("Suitability Score: {suitablity_score} %"))

}

def create_deck(hex_layer, tooltip):
    x, y = (77.06463665845695, 28.437398083125167)
    view_state = pdk.ViewState(latitude=y, longitude=x, zoom=12, bearing=0, pitch=45)
    # Render
    r = pdk.Deck(layers=[hex_layer], initial_view_state=view_state, tooltip=tooltip)

    return r


st.sidebar.markdown("## Map Controls")

parameter = st.sidebar.selectbox(
    label="Parameter",
    options=options.keys(),
)

hex_layer, tooltip = options[parameter]
deck = create_deck(hex_layer, tooltip)

st.pydeck_chart(deck, use_container_width=False)

Overwriting app.py


In [ ]:
password = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")
print("Password/Enpoint IP for localtunnel is:", password)

!streamlit run /content/app.py &>/content/logs.txt &
!npx localtunnel --port 8501

Password/Enpoint IP for localtunnel is: 34.75.45.42
npx: installed 22 in 2.986s
your url is: https://chatty-ends-unite.loca.lt


## TODO

- ~Add the ranking summary plot~
- Add legend for each plot
- Try full screen for the plot